In [1]:
import numpy as np
import pandas as pd
import plotly.express as px
import os
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
print(os.environ.get("DATA_SOURCE_DIR"))

data/


In [3]:
DATA_SOURCE = "../data/5_yr_data/"

In [4]:
up_5yrs = pd.read_csv(DATA_SOURCE+"UP5_years.csv")

In [5]:
up_5yrs.head()

,Unnamed: 0,district_name,market_name,commodity,variety,grade,min_rs_quintal,max_rs_quintal,modal_rs_quintal,date,year,month,day_of_month
0,446185,Auraiya,Achalda,Potato,Badshah,FAQ,600.0,700.0,670.0,04 Oct 2019,2019,Oct,4
1,446186,Auraiya,Achalda,Potato,Badshah,FAQ,780.0,820.0,800.0,05 Dec 2019,2019,Dec,5
2,446187,Auraiya,Achalda,Potato,Badshah,FAQ,850.0,920.0,900.0,17 Dec 2019,2019,Dec,17
3,446188,Auraiya,Achalda,Potato,Badshah,FAQ,890.0,950.0,900.0,18 Oct 2019,2019,Oct,18
4,446189,Auraiya,Achalda,Potato,Badshah,FAQ,1000.0,1090.0,1060.0,29 Jan 2020,2020,Jan,29


In [6]:
up_5yrs.drop(columns=['Unnamed: 0'], inplace=True)

In [7]:
up_5yrs.head()

,district_name,market_name,commodity,variety,grade,min_rs_quintal,max_rs_quintal,modal_rs_quintal,date,year,month,day_of_month
0,Auraiya,Achalda,Potato,Badshah,FAQ,600.0,700.0,670.0,04 Oct 2019,2019,Oct,4
1,Auraiya,Achalda,Potato,Badshah,FAQ,780.0,820.0,800.0,05 Dec 2019,2019,Dec,5
2,Auraiya,Achalda,Potato,Badshah,FAQ,850.0,920.0,900.0,17 Dec 2019,2019,Dec,17
3,Auraiya,Achalda,Potato,Badshah,FAQ,890.0,950.0,900.0,18 Oct 2019,2019,Oct,18
4,Auraiya,Achalda,Potato,Badshah,FAQ,1000.0,1090.0,1060.0,29 Jan 2020,2020,Jan,29


In [8]:
up_5yrs.shape

(1226694, 12)

## Missing Values

In [9]:
up_5yrs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1226694 entries, 0 to 1226693
Data columns (total 12 columns):
 #   Column            Non-Null Count    Dtype  
---  ------            --------------    -----  
 0   district_name     1226694 non-null  object 
 1   market_name       1226694 non-null  object 
 2   commodity         1226694 non-null  object 
 3   variety           1226694 non-null  object 
 4   grade             1226694 non-null  object 
 5   min_rs_quintal    1226694 non-null  float64
 6   max_rs_quintal    1226694 non-null  float64
 7   modal_rs_quintal  1226694 non-null  float64
 8   date              1226694 non-null  object 
 9   year              1226694 non-null  int64  
 10  month             1226694 non-null  object 
 11  day_of_month      1226694 non-null  int64  
dtypes: float64(3), int64(2), object(7)
memory usage: 112.3+ MB


### I think we have populated the missing data with zero and so the above command says that there is no missing value
### Need to check where we are populating with zero when data is not present

In [10]:
up_5yrs.describe()

,min_rs_quintal,max_rs_quintal,modal_rs_quintal,year,day_of_month
count,1.226694e+06,1.226694e+06,1.226694e+06,1.226694e+06,1.226694e+06
mean,1.704229e+03,1.821545e+03,1.772429e+03,2.020855e+03,1.572314e+01
std,7.698786e+02,7.849455e+02,7.662350e+02,1.934080e+00,8.784076e+00
min,0.000000e+00,0.000000e+00,1.100000e+01,2.018000e+03,1.000000e+00
25%,1.075000e+03,1.200000e+03,1.150000e+03,2.019000e+03,8.000000e+00
50%,1.725000e+03,1.830000e+03,1.785000e+03,2.021000e+03,1.600000e+01
75%,2.280000e+03,2.400000e+03,2.340000e+03,2.023000e+03,2.300000e+01
max,1.050000e+04,2.650000e+04,1.054000e+04,2.024000e+03,3.100000e+01


In [11]:
def get_outlier_data(field, df):
    outlier_data = []
    unique_commodities = set(df.commodity.values)

    for commodity in unique_commodities:
        min_rs = df[df['commodity'] == commodity][field]
        
        commodity_values = {
            'commodity': commodity,
            'min_value': min_rs.min(),
            '5th_percentile': min_rs.quantile(0.05),
            '25th_percentile': min_rs.quantile(0.25),
            '50th_percentile': min_rs.quantile(0.50),
            '75th_percentile': min_rs.quantile(0.75),
            '90th_percentile': min_rs.quantile(0.90),
            '95th_percentile': min_rs.quantile(0.95),
            '99th_percentile': min_rs.quantile(0.99),
            '99.5th_percentile': min_rs.quantile(0.995),
            'max_value': min_rs.max()
        }
        
        outlier_data.append(commodity_values)

    df_outliers = pd.DataFrame(outlier_data)
    return df_outliers


In [12]:
get_outlier_data('min_rs_quintal', up_5yrs)

,commodity,min_value,5th_percentile,25th_percentile,50th_percentile,75th_percentile,90th_percentile,95th_percentile,99th_percentile,99.5th_percentile,max_value
0,Onion,0.0,750.0,1100.0,1450.0,2050.0,2900.0,3400.0,4442.6,5000.000,10500.0
1,Rice,0.0,2050.0,2300.0,2440.0,2600.0,2850.0,2950.0,3300.0,4715.700,9800.0
2,Wheat,0.0,1570.0,1750.0,1900.0,2150.0,2370.0,2450.0,2650.0,2710.000,7020.0
3,Potato,0.0,450.0,700.0,945.0,1290.0,1700.0,2200.0,2900.0,3009.175,7400.0


In [13]:
get_outlier_data('max_rs_quintal', up_5yrs)

,commodity,min_value,5th_percentile,25th_percentile,50th_percentile,75th_percentile,90th_percentile,95th_percentile,99th_percentile,99.5th_percentile,max_value
0,Onion,0.0,850.0,1250.0,1600.0,2200.0,3100.0,3600.0,4650.0,5250.00,11050.0
1,Rice,0.0,2185.0,2400.0,2550.0,2725.0,3000.0,3100.0,3480.0,4797.85,26500.0
2,Wheat,0.0,1640.0,1840.0,1975.0,2270.0,2460.0,2550.0,2750.0,2810.00,16700.0
3,Potato,0.0,550.0,800.0,1060.0,1400.0,1850.0,2360.0,3050.0,3200.00,11200.0


In [24]:
get_outlier_data('modal_rs_quintal', up_5yrs)

,commodity,min_value,5th_percentile,25th_percentile,50th_percentile,75th_percentile,90th_percentile,95th_percentile,99th_percentile,99.5th_percentile,max_value
0,Onion,88.0,800.0,1190.0,1530.0,2140.0,3000.0,3500.0,4500.0,5100.0,10540.0
1,Rice,21.0,2135.0,2355.0,2500.0,2665.0,2940.0,3005.0,3380.0,4750.0,9820.0
2,Wheat,17.0,1620.0,1800.0,1935.0,2220.0,2415.0,2500.0,2700.0,2760.0,7040.0
3,Potato,11.0,510.0,760.0,1000.0,1350.0,1775.0,2300.0,2970.0,3100.0,9050.0


In [31]:
up_5yrs[(up_5yrs['modal_rs_quintal'] > 5100) & (up_5yrs['commodity'] == 'Onion')]['year'].value_counts()

year
2020    1136
2019     456
2023      58
2021      10
2018       3
2022       3
2024       2
Name: count, dtype: int64

In [66]:
up_5yrs[(up_5yrs['modal_rs_quintal'] > 3000) & (up_5yrs['commodity'] == 'Wheat') & (up_5yrs.year == 2023) & (up_5yrs.month == 'Jan')]

,district_name,market_name,commodity,variety,grade,min_rs_quintal,max_rs_quintal,modal_rs_quintal,date,year,month,day_of_month
593535,Meerut,Meerut,Wheat,Dara,FAQ,2920.0,3100.0,3015.0,27 Jan 2023,2023,Jan,27
609596,Meerut,Meerut,Wheat,Dara,FAQ,2925.0,3150.0,3010.0,25 Jan 2023,2023,Jan,25
616177,Gonda,Gonda,Wheat,Dara,FAQ,3900.0,3980.0,3940.0,19 Jan 2023,2023,Jan,19


### From the above data, it is clear that the modal_price > 5100 was pre-dominantly during Covid Period. So we would be capping it to 5100.

In [47]:
# up_5yrs[(up_5yrs['modal_rs_quintal'] > 4750) & (up_5yrs['commodity'] == 'Rice')]['year'].value_counts()
# up_5yrs[(up_5yrs['modal_rs_quintal'] > 5000) & (up_5yrs['commodity'] == 'Rice')]['district_name'].value_counts()
up_5yrs[(up_5yrs['variety'] == 'Fine(Basmati)') & (up_5yrs['commodity'] == 'Rice')]['district_name'].value_counts()

district_name
Gautam Budh Nagar    791
Fatehpur              44
Hathras               10
Ayodhya                1
Name: count, dtype: int64

### We clearly see from the data above that the `Fine(Basmati)` vareity of rice mainly sold in `Gautam Budh Nagar` district is having clearly more than double the price of the other vareities available

### To do proper prediction we will initially not include the `Fine(Basmati)` in the model as it may skew the learnings of th model but later we may try including by possibly using hyperparameters

### We could clearly see that there are outliers in all the chosen categories

In [19]:
up_5yrs[(up_5yrs['commodity'] == "Onion") & (up_5yrs['max_rs_quintal'] > 5250)]['year'].value_counts()

year
2020    1108
2019     465
2023      57
2021       8
2018       7
2022       3
2024       2
Name: count, dtype: int64

In [20]:
up_5yrs[(up_5yrs['commodity'] == "Rice") & (up_5yrs['max_rs_quintal'] > 4798)]['year'].value_counts()

year
2021    266
2020    238
2022    231
2023    182
2024    145
2019     45
2018     15
Name: count, dtype: int64

In [22]:
up_5yrs[(up_5yrs['commodity'] == "Potato") & (up_5yrs['max_rs_quintal'] > 3200)]['year'].value_counts()

year
2020    1470
2024      49
2019      15
2021       9
2018       8
2022       2
2023       2
Name: count, dtype: int64

In [23]:
up_5yrs[(up_5yrs['commodity'] == "Wheat") & (up_5yrs['max_rs_quintal'] > 2810)]['year'].value_counts()

year
2023    1299
2022      82
2024      36
2020      19
2018      11
2019      10
2021       8
Name: count, dtype: int64